# Aim

This is the basic loop used to compile the flare catalog. Feel free to use it replacing the path to the directory containing the folder hosting your lightcurves.

In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import toffee
import warnings
import os
import re

In [49]:
periodogram = [0.01,10]

window_length = 0.25

one_sigma_percentile = 84

magic_flare_std = 3.0

warnings.filterwarnings("ignore")
#great, now we just need to do it for all our stars and sort their spectral types

path = 'import path'

flare_characteristics = np.array([[],[],[],[],[],[],[],[],[]])

TIC_ids = np.array([])

#hold the times for all the primary double flares
sympathetic_flare_times = []

#and which TIC they come from
sympathetic_flare_tics = []

#array to hold the lightcurve with negative fluxes
negative_sectors = []

#array to hold the lightcurves with a lot of breaks
broken_sectors = []

#array to hold the lightcurves for which we couldn't fit a double sine
#plotting parameters
flare_color = 'dodgerblue'
fontsize = 24
labelsize = 18


i = 0
    
for name in os.listdir(path):

    #sometimes bunk files get caught up in the directory so we're gonna check to see
    #if the file starts with 'P'

    if name[0] == 'S':

        print(name)
        
        #import light curve from folder
        light_curve = np.genfromtxt(path + name, delimiter = ',')
    
        #Find TIC number

        TIC_number = int(re.findall("(\d+)", name)[0])

        ##########FINDING SECTOR NUMBER OF LIGHTCURVE########


        #pull out second number from file name, telling the TESS Sector this
        #light curve comes from

        TESS_sector = int(re.findall("(\d+)", name)[1])

        #unpack array
        
        time = light_curve[0]
    
        flux = light_curve[1]
    
        quality = light_curve[2]

        flux_err = light_curve[3]

        #find flares


        flare_results = toffee.flare_finder(time, flux, flux_err, quality, visualize_fit = False,
                                     rise_func = 'gaussian', consecutive = False,
                                     magic_flare_std = 3.0, sec_flare_std = 2.0, 
                                     periodogram = periodogram, clip_breaks = 100,
                                     min_break = 0.025, fit_twice = True, fit_multiple_secs = False,
                                     detrend = True, flag_values = [0])

        #cut out the crap points using quality flags
    
        perfect_points = quality == 0
    
        #unpack good values of flux and time for the star
    
        flux = flux[perfect_points]
    
        time = time[perfect_points][np.isnan(flux) == False]

        flux_err = flux_err[perfect_points][np.isnan(flux) == False]
    
        flux = flux[np.isnan(flux) == False]

        #check that it doesn't dip below zero
        #if there are points in the lightcurve with fluxes less than zero (due to poorly reduced lightcurves
        #with eclipsing binaries) then it'll not work in the detrending and is skipped

        if min(flux) <= 0:

            #we have a sector with negative flux, save for later                

            negative_sectors.append(str(TIC_number) + '_' + str(TESS_sector))

            print('Negative!')

            continue

        #run the flattening curve to see if something else can break the LS test

        t, lc_quad, lc_wotan, lc_flat, periodic = toffee.flatten(time, flux, flux_err, plot_results=False,
                                                    short_window=0.25, periodogram = periodogram)

        if type(t) == str:

            #we have some problematic sector

            broken_sectors.append(str(TIC_number) + '_' + str(TESS_sector))

            print('Broken!')

            continue


        #or if the fit didn't converge it was returned as false

        if type(t) == bool:

            #we have some problematic sector

            broken_sectors.append(str(TIC_number) + '_' + str(TESS_sector))

            print('RunTime Error!')

            continue

    
        #find the flare peak times

        flare_peak_times = flare_results[0]

        #and their amps

        flare_amps = flare_results[3]

        #sort them in ascending order

        flare_amps = flare_amps[np.argsort(flare_peak_times)]

        flare_peak_times = np.sort(flare_peak_times)

        #also keep all flares
                
        flare_characteristics = np.append(flare_characteristics, flare_results, axis = 1)

        #and keep ther TICs

        #append the tic number, we want to add as many
        #iterations of the tics as we do flares from this sector
 
        TIC_ids = np.append(TIC_ids, np.full(len(flare_amps), int(TIC_number)))

        i+=1
        print(str(i) +'/9,274')

SPOC_Light_Curves_84336253_43.csv
1/9,274
SPOC_Light_Curves_360261300_11.csv
2/9,274
SPOC_Light_Curves_309403283_35.csv
3/9,274
SPOC_Light_Curves_397353968_44.csv
4/9,274
SPOC_Light_Curves_453684455_12.csv
5/9,274
SPOC_Light_Curves_118679163_43.csv
6/9,274
SPOC_Light_Curves_265179710_12.csv
7/9,274
SPOC_Light_Curves_114061043_44.csv
8/9,274
SPOC_Light_Curves_125754787_44.csv
9/9,274
SPOC_Light_Curves_453095065_5.csv
10/9,274
SPOC_Light_Curves_258066901_42.csv
11/9,274
SPOC_Light_Curves_285555777_45.csv
12/9,274
SPOC_Light_Curves_125735913_42.csv
13/9,274
SPOC_Light_Curves_75827390_62.csv
14/9,274
SPOC_Light_Curves_308667775_8.csv
15/9,274
SPOC_Light_Curves_220425740_65.csv
16/9,274
SPOC_Light_Curves_166527623_38.csv
17/9,274
SPOC_Light_Curves_361815724_66.csv
18/9,274
SPOC_Light_Curves_67787307_44.csv
19/9,274
SPOC_Light_Curves_464898410_10.csv
20/9,274
SPOC_Light_Curves_46633710_44.csv
21/9,274
SPOC_Light_Curves_293970607_67.csv
22/9,274
SPOC_Light_Curves_67318521_64.csv
23/9,274
SPOC

In [50]:
print('We found ' + str(len(TIC_ids)) + ' flares')

print('With ' + str(len(sympathetic_flare_times)) + ' double flares')

print('There are ' + str(len(broken_sectors) + len(negative_sectors)) + ' that broke the detrending')

print('')

print('Of these ' + str(len(broken_sectors)) + ' just broke the code')

print('')

print(str(len(negative_sectors)) + ' had negative flux')

We found 19416 flares
With 0 double flares
There are 54 that broke the detrending

Of these 0 just broke the code

54 had negative flux


In [53]:
#Add TIC to Flare Catalog

flare_catalog = pd.DataFrame(flare_characteristics.T, columns = ['T_peak', 'T_start', 'T_end', 'Amplitude', 'ED',
                                                                'Primary_or_Secondary', 'Num_Points', 'Num_Abv_Threshold',
                                                                'Amp_Sigma'])

flare_catalog['TIC'] = TIC_ids

flare_catalog = flare_catalog.iloc[:, [9, 0, 1, 2, 3, 4, 5, 6, 7, 8]]

flare_catalog

path = 'export path'

flare_catalog.to_csv(path + 'Flare Catalog.csv')